In [70]:
import cv2
import numpy as np
from skimage import feature
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2 
from skimage.feature import local_binary_pattern
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import euclidean_distances
import pyswarms as ps
from pyswarms.discrete import BinaryPSO


In [71]:
# load dataset
with open('X-Train.npy', 'rb') as f:
    X_train = np.load(f)
    
with open('Y-Train.npy', 'rb') as f:
    Y_train = np.load(f)
    
with open('X-Test.npy', 'rb') as f:
    X_test = np.load(f)
    
with open('Y-Test.npy', 'rb') as f:
    Y_test = np.load(f)
    

In [72]:
# Create an instance of the classifier
classifier = KNeighborsClassifier(n_neighbors=3,metric="cosine")

# Define objective function for each particle
def ParticleFitness(m):
    total_features = 256
    if np.count_nonzero(m) == 0:
        X_subset_train = X_train
        X_subset_test = X_test
    else:
        X_subset_train = X_train[:,m==1]
        X_subset_test = X_test[:,m==1]
    
    classifier.fit(X_subset_train, Y_train)
    P = classifier.score(X_subset_test,Y_test)
    # Compute for the objective function
    #j = (alpha * (1.0 - P) + (1.0 - alpha) * (1 - (X_subset_train.shape[1] / total_features)))

    return 1-P

In [73]:
def Fitness(x, alpha=0.88,**k):
    n_particles = x.shape[0]
    j = [ParticleFitness(x[i]) for i in range(n_particles)]
    return np.array(j)

In [74]:
# Initialize swarm, arbitrary
options = {'c1': 1.4, 'c2': 1.4, 'w':0.72, 'k': 20, 'p':2}


# Call instance of PSO
dimensions = 256 # dimensions should be the number of features
optimizer = ps.discrete.BinaryPSO(n_particles=20, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(Fitness,iters=150, verbose=2)
optimizer.reset()

2022-02-15 18:06:22,094 - pyswarms.discrete.binary - INFO - Optimize for 150 iters with {'c1': 1.4, 'c2': 1.4, 'w': 0.72, 'k': 20, 'p': 2}
pyswarms.discrete.binary: 100%|██████████████████████████████████████████████████████████████|150/150, best_cost=0.0976
2022-02-15 18:06:41,566 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.09756097560975607, best pos: [0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 1 0 1 1 0
 0 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0
 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0
 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0
 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1
 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 1
 0 1 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0]


In [75]:
classifier = KNeighborsClassifier(n_neighbors=3,metric="cosine")


X_selected_features_train = X_train[:,pos==1]  
X_selected_features_test = X_test[:,pos==1]

classifier.fit(X_selected_features_train, Y_train)

subset_performance = (classifier.predict(X_selected_features_test) == Y_test).mean()
print('Subset performance: %.3f' % (subset_performance))

Subset performance: 0.902


In [76]:
print("Count of selected Features:",sum(pos))

Count of selected Features: 134


In [77]:

classfier = KNeighborsClassifier(n_neighbors=3,metric="cosine")

X_selected_features_train = X_train.copy()  
X_selected_features_test = X_test.copy()

classifier.fit(X_selected_features_train, Y_train)

# Compute performance
subset_performance = (classifier.predict(X_selected_features_test) == Y_test).mean()


print('Actual features performance: %.3f' % (subset_performance))

Actual features performance: 0.854


In [78]:
with open('PSO-LBP-Positions.npy', 'wb') as f:
    np.save(f, pos)